# 1.Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import norm
import missingno as mno
sns.set_style("whitegrid", {"grid.color": ".2", "grid.linestyle": ":"})

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFECV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score, GridSearchCV
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# 2.Load and Show Basic Data Info

In [ ]:
#Get dataset directory
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# Load the data
housing = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
data = housing.copy()

#show all columns & rows
pd.set_option('display.max_columns', None)  
pd.set_option('display.max_rows', None)  

# Show the first 5 rows
data.head()

This dataset contains 81 columns. The first column is the ID, and the last column is our target variable'SalePrice.' The remaining columns are potential features that we need to analyze further to determine which ones might be helpful in predicting 'SalePrice.'

In [ ]:
# Show the basic information of the data
data.info()

This dataset consists of 1,460 samples，and contains 43 features of object type (typically strings or categories), 35 features of integer type, and 3 features of float type.

# 3.Exploratory Data Analysis

## 3.1.Target Distribution

In [ ]:
# Plot the distribution of 'LotFrontage' values
plt.figure(figsize=(7, 3))
sns.distplot(data['SalePrice'],fit = norm)
plt.show()

The distribution of target appears to be right-skewed, meaning that most houses have saleprices concentrated in a smaller range, while larger values are less common. 

## 3.2.Correlation Matrix

In [ ]:
# Compute the correlation matrix
corr_matrix = data.corr()

# Set up the matplotlib figure
plt.figure(figsize=(12, 9))

# Draw the heatmap
sns.heatmap(corr_matrix, vmax=.8, square=True, cmap='coolwarm')
plt.show()

In [ ]:
corr_matrix['SalePrice'].sort_values(ascending=False)

### 3.2.1.Features with high corr to target

In [ ]:
corr_matrix = data.corr()

#Set high correlation threshold
thres = 0.5

# Find features that have a correlation with 'SalePrice' greater than 0.5
high_corr_cols = corr_matrix[abs(corr_matrix['SalePrice']) > thres].index.tolist()
high_corr_features = [col for col in high_corr_cols if (col != 'SalePrice')]

print('Features with high correlations to target are:')
print(high_corr_features)

Visualize Features with high correlation to target

In [ ]:
#Visualize Correlation 
plt.figure(figsize=(10,10))
g = sns.heatmap(data[high_corr_cols].corr(),annot=True,cmap="RdYlGn")

### 3.2.2.Features with low corr to target

In [ ]:
#Set low correlation threshold
thres = 0.1

# Find features that have a correlation with 'SalePrice' greater than 0.5
low_corr_cols = corr_matrix[abs(corr_matrix['SalePrice']) < thres].index.tolist()
low_corr_features = [col for col in low_corr_cols if (col != 'SalePrice')]

print('Features with low correlations to target are:')
print(low_corr_features)

# 4.Data Preprocessing

## 4.1.Delete Useless Columns
We can start by removing columns that do not contribute to predicting 'SalePrice.' For example, the 'Id' column is simply an identifier for each sample and does not provide any helpful information for predicting 'SalePrice.' Therefore, we can delete it from the dataset.

In [ ]:
# Drop the 'Id' column
data = data.drop('Id', axis=1)

## 4.2.Covert Data Type
According to data description, MSSubClass Identifies the type of dwelling involved in the sale. It's current dtype is float64, its values indicating types.

In [ ]:
# convert to object
data['MSSubClass'] = data['MSSubClass'].astype('object')

## 4.3.Remove Duplicates
We need to check if there are any duplicate rows in the dataset. If duplicates are found, we should remove them because duplicate data can have a negative impact on our model.

In [ ]:
# Check for duplicates
duplicates = data.duplicated().sum()

duplicates

It's a good news that the dataset does not have any duplicate rows. So, We don't need to handle duplicate data.

## 4.4.Missing Values
Let's handle missing values by first looking at the number of missing values in each column. This can help us determine how to handle these missing values.

### 4.4.1.Missing Value Analysis

In [ ]:
# Check for missing values
total = data.isnull().sum().sort_values(ascending=False)
total = total[total > 0]
percent = (data.isnull().sum() / data.shape[0] * 100).round(2).sort_values(ascending=False)
percent = percent[percent > 0] 

dtypes = data.dtypes[total.index]
result = pd.concat([total, percent, dtypes], axis=1, keys=['Total', 'Percent', 'Dtype'])
result

We can see that some features have a large number of missing values, such as 'PoolQC,' 'MiscFeature,' 'Alley,' and 'Fence.' Most of the values in these features are missing, so we can consider directly deleting these features.

In [ ]:
#Visualizing columns with missing values
mno.matrix(data.loc[:, data.isna().any()], color = '0.2')

Looks like some values are not missing at random, such as 'MasVnr', 'Bsmt', and'Garage'related features

### 4.4.2.Treat Missing Values for Each Feature

#### 1.LotFrontage
Linear feet of street connected to property

* Numerical feature
* Values missing at random
* Missing percentage: 17.74%

The distribution of 'LotFrontage' appears to be right-skewed, meaning that most houses have street frontage lengths concentrated in a smaller range, while larger values are less common. In this case, impute missing values with median may be better than using the mean, as the median is more robust to outliers and skewed distributions.

In [ ]:
# Plot the distribution of 'LotFrontage' values
plt.figure(figsize=(5, 3))
sns.distplot(data['LotFrontage'], fit = norm)
plt.show()

In [ ]:
#Impute with median
data['LotFrontage'].fillna(data['LotFrontage'].median(), inplace = True)

#### 2.Alley
Type of alley access to property

* Categorical feature
* Values not missing at random
* Mssing percentage: 93.77%

Missing values indicate no alley at the property, so impute with 'None'.

In [ ]:
#show unique values
data['Alley'].value_counts(dropna = False)

In [ ]:
#impute with 'None'
data['Alley'].fillna('None', inplace = True)

#### 3.MasVnr...
MasVnrType: Masonry veneer type\
MasVnrArea: Masonry veneer area in square feet

* Categorical feature & numerical feature
* Values not missing at random
* Missing percentage: 0.55% & 0.55%
* All missing values appear simultaneously

According to the mono plot, it seems that the missing values for these two features occur simultaneously. 'MasVnrType' represents the type of masonry veneer, while 'MasVnrArea' represents the area of masonry veneer. If the missing values for both of these features occur together, it could mean that these houses do not have any masonry veneer. Therefore, when the missing values occur simultaneously, we impute 'MasVnrType' with 'None' and 'MasVnrArea' with 0.

In [ ]:
#show unique values of MasVnrType
data['MasVnrType'].value_counts(dropna = False)

In [ ]:
# When missing values appears simultaneously, impute 'MasVnrType' with 'NA' and 'MasVnrArea' with 0.
data.loc[data['MasVnrType'].isnull() & data['MasVnrArea'].isnull(), ['MasVnrType','MasVnrArea']] = ['None',0.0]

#### 4.Bsmt...
BsmtQual: Evaluates the height of the basement\
BsmtCond: Evaluates the general condition of the basement\
BsmtExposure: Refers to walkout or garden level walls\
BsmtFinType1: Rating of basement finished area\
BsmtFinType2: Rating of basement finished area (if multiple types)

* All are Categorical features
* <b>Some values not missing at random</b>
* Missing percentage: 2.60% for 'BsmtExposure' and 'BsmtFinType2'; 2.53% for 'BsmtQual','BsmtCond', and 'BsmtFinType1'
* Some values missing simultanouesly

According to the mono plot, it seems that the missing values for Bsmt related features occur simultaneously. Moreover, we can always find these simultaneous missings at 'TotalBsmtSF' = 0, which could mean that these houses do not have basement. Therefore, at 'TotalBsmtSF' = 0, impute Bsmt related features with 'None'.

In [ ]:
#creat a list for Bsmt related features
cols = ['BsmtQual','BsmtCond','BsmtFinType1', 'BsmtExposure', 'BsmtFinType2']

# impute with 'None' for features not missing at random (TotalBsmtSF = 0)
data.loc[data['TotalBsmtSF'] == 0, cols] = data.loc[data['TotalBsmtSF'] == 0, cols].fillna('None')

#show unique values
for i in cols:
    print(data[i].value_counts(dropna = False), '\n')

* <b>Some values missing at random</b>\
However, we still have missing values for 'BsmtExposure' and 'BsmtFinType2' after imputation. This is because now TotalBsmtSF != 0, so houses do have basements. So, values are missing at random. So, impute remaining missing values with mode since 'BsmtExposure' and 'BsmtFinType2' are categorical features.

In [ ]:
#impute remaining missing values with mode
data['BsmtExposure'].fillna(data['BsmtExposure'].mode()[0], inplace = True)
data['BsmtFinType2'].fillna(data['BsmtFinType2'].mode()[0], inplace = True)

#### 5.Electrical
Electrical system

* Categorical feature
* Values missing at random
* Missing percentage: 0.07%

The data description did not expect to have missing values for this feature. So I guess that this could happen either because it's a record mistake or because this property does not have electrical system indeed. 

If it's a record mistake, we should impute it with mode, but if this property does not have electrical system indeed, it should be treated as an outlier. For our analysis, we assume it's a record mistake.

In [ ]:
data['Electrical'].value_counts(dropna = False)

In [ ]:
data['Electrical'].fillna(data['Electrical'].mode()[0], inplace = True)

#### 6.FireplaceQu
Fireplace quality

* Categorical feature
* Values not missing at random
* Missing percentage: 47.26%

Missing values indicate no FireplaceQu at the property, so impute with 'None'.

In [ ]:
#show values
data['FireplaceQu'].value_counts(dropna = False)

In [ ]:
#impute with 'None'
data['FireplaceQu'].fillna('None', inplace = True)

#### 7.Garage...
-GarageType: Garage location\
-GarageFinish: Interior finish of the garage\
-GarageQual: Garage quality\
-GarageCond: Garage condition\
-GarageYrBlt: Year garage was built

* Values not missing at random
* Categorical features; GarageYrBlt is numerical
* Missing percentages are: 5.55%

According to the mono plot, it seems that the missing values for Garage related features occur simultaneously.Moreover, we can always find these simultaneous missing at 'GarageArea' = 0, which could mean that these houses do not have garage. Therefore, at 'GarageArea' = 0, we impute garage related features with 'None'.

In [ ]:
# Plot the distribution of 'LotFrontage' values
plt.figure(figsize=(5, 3))
sns.distplot(data['GarageYrBlt'])
plt.show()

It seems there are two central tendency of the distribution: one before 1980s and one after 1980. Since missing values indicate no garage, if we impute with 'None', we are going to have to dtypes for this features. So, we perform feature creation for GarageYrBlt 

In [ ]:
#creat a list for Garage related features
cols = ['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'GarageYrBlt']

# impute with 'None' for features not missing at random (GarageArea = 0)
data.loc[data['GarageArea'] == 0, cols] = data.loc[data['GarageArea'] == 0, cols].fillna('None')

In [ ]:
#Feature Creation 
data['GarageYrBlt'] = data['GarageYrBlt'].apply(lambda x: 'None' if x == 'None' else ('New' if x > 1980 else 'Old'))

In [ ]:
data['GarageYrBlt'].value_counts()

#### 8.PoolQC
Pool quality

* Categorical Feature
* Values not missing at random
* Missing percentage: 99.52%

Missing values indicate no pool at the property.

In [ ]:
#show values
data['PoolQC'].value_counts(dropna = False)

In [ ]:
#impute with 'None'
data['PoolQC'].fillna('None', inplace = True)

#### 9.Fence
Fence quality

* Categorical Feature
* Values not missing at random
* Missing percentage: 80.75%

Fence means fence quality-type,missing values indicate no fence at the property. However, at this point we don't know for sure how fence influences the target. So, let's impute it with 'None' for now and review it later in feature engineering.

In [ ]:
#show unique values
data['Fence'].value_counts(dropna = False)

In [ ]:
#impute with 'None'
data['Fence'].fillna('None', inplace = True)

#### 10.MiscFeature
Miscellaneous feature not covered in other categories

* Categorical Feature
* Values not missing at random
* Missing percentage:96.30%

Missing values indicate no MiscFeature at the property, so impute with 'None'.

In [ ]:
#show unique values
data['MiscFeature'].value_counts(dropna = False)

In [ ]:
#impute with 'None'
data['MiscFeature'].fillna('None', inplace = True)

## 4.5.Check for Remaining Missing Values

In [ ]:
data.isnull().any().any()

So far, no missing value remains. Let's proceed to treating outliers

## 4.6.Feature Encoding

### 4.6.1.Ordinal Mapping
perform ordinal mapp for ordered features

* 10 - very excellent
* 9 - excellent (Ex)
* 8 - very good
* 7 - good (Gd)
* 6 - above average
* 5 - average (TA)
* 4 - below average
* 3 - fair (Fa)
* 2 - poor (Po)
* 1 - very poor
* 0 - None

In [ ]:
data['GarageQual'].unique()

In [ ]:
# Identify categorical features in the dataset
cat_features = data.select_dtypes(include=['object']).columns.tolist()

# Define the ordered values
ordered_values = ['Ex', 'Gd', 'TA', 'Fa', 'Po']

# Initialize an empty list to hold the features that contain ordered values
ordinal_features = []

# Find features with ordered values
for i in cat_features:
    
    if any(value in data[i].unique() for value in ordered_values):
        ordinal_features.append(i)

print('Ordinal features are:')        
print(ordinal_features)

In [ ]:
#check unique values for these features
for i in ordinal_features:
    print(data[i].unique())

In [ ]:
#drop 'BsmtExposure'
ordinal_features_1 = [cols for cols in ordinal_features if (cols != 'BsmtExposure')]
print(ordinal_features_1)

In [ ]:
# conduct ordinal encoding for these features
for i in ordinal_features_1:
    data[i] = data[i].map({'Ex':9, 'Gd':7, 'TA':5, 'Fa':3, 'Po':2, 'None':0 })

### 4.6.2.Binary Mapping
perform binary mapping for binary features

In [ ]:
#binary variables are defined as number of unique values = 2 
bin_features = [col for col in data.columns if data[col].nunique() == 2]
bin_features

In [ ]:
# Applying the binary mapping to binary columns
data['Street'] = data['Street'].map({'Pave': 1, "Grvl": 0})
data['Utilities'] = data['Utilities'].map({'AllPub': 1, "NoSeWa": 0})
data['CentralAir'] = data['CentralAir'].map({'Y': 1, "N": 0})

## 4.7.Treat Outliers

Now that we have performed feature encoding for ordinal & binary features, it is now appropriate to distinguish between discrete and continous features for outliers treatment. 

### Numerical Features

In [ ]:
#find out all numerical features
num_features = [cols for cols in data.select_dtypes(include = np.number).columns if cols != 'SalePrice']
print('Numerical features are:')
print(num_features)

### Discrete Features

In [ ]:
# Get the number of unique values for each numerical feature
num_unique_counts = data[num_features].nunique().sort_values()

# Identify discrete features
discrete_features = num_unique_counts[num_unique_counts < 30].index.tolist()

print('Discrete Features are:')
print(discrete_features)

### Continous Features

In [ ]:
continous_features = [cols for cols in num_features if (cols not in discrete_features)]

print('Continous Features are:')
print(continous_features)

### Continous Features with Extreme Outliers

In [ ]:
# Find all features where the maximum value is greater than three times the third quartile
cf_with_extremo = []

for i in continous_features:
    
    thres = 3 
    
    Q1 = data[i].quantile(0.25)
    Q3 = data[i].quantile(0.75)
    min_val = data[i].min()
    max_val = data[i].max()
    
    if (max_val > thres*Q3 and Q3 != 0) or (min_val < Q1/thres):
        cf_with_extremo.append(i)
        
print('Continous features with extreme outliers are:')
print(cf_with_extremo)

In [ ]:
data[cf_with_extremo].describe(percentiles = [0.003,0.01, 0.99, 0.997])

### Box Plot

In [ ]:
#Function for visualizing continous features, box plot
def box_plot(x):
    ax = sns.boxplot(y = data[x], color = 'darkcyan', showfliers = True, showmeans = True, 
                     meanprops={"marker":"s","markerfacecolor":"white", "markeredgecolor":"crimson"})
    ax.set_ylabel('')
    ax.set_title('{}'.format(x), fontsize = 14, fontweight = 'bold', pad = 5)
    ax.patch.set_edgecolor('black')
    ax.patch.set_linewidth(1.5)

In [ ]:
plt.figure(figsize = [14,40])
for i in range(len(cf_with_extremo)):
    plt.subplot(10,5,i+1)
    box_plot(cf_with_extremo[i])
plt.tight_layout()

### Remove Outliers and Visualize Effects on a Copy

In [ ]:
data1 = data.copy()

In [ ]:
data1 = data.copy()

#Function for visualizing continous features, box plot
def box_plot(x):
    ax = sns.boxplot(y = data1[x], color = 'darkcyan', showfliers = True, showmeans = True, 
                     meanprops={"marker":"s","markerfacecolor":"white", "markeredgecolor":"crimson"})
    ax.set_ylabel('')
    ax.set_title('{}'.format(x), fontsize = 14, fontweight = 'bold', pad = 5)
    ax.patch.set_edgecolor('black')
    ax.patch.set_linewidth(1.5)

In [ ]:
data1 = data1[data1['LotFrontage'] < 300]
data1 = data1[data1['LotArea'] < 70000] 
data1 = data1[data1['MasVnrArea'] < 1200]
data1 = data1[data1['WoodDeckSF'] < 800]
data1 = data1[data1['BsmtUnfSF'] < 2300]

In [ ]:
data1.shape

In [ ]:
plt.figure(figsize = [18,32])
for i in range(len(cf_with_extremo)):
    plt.subplot(5,5,i+1)
    box_plot(cf_with_extremo[i])
plt.tight_layout()

### Apply Treatment to Preprocessing Data 
When we are satisfied with the box plot after removing outliers, we apply it to data we are preprocessing.

In [ ]:
data = data1
data.shape

Before outlier treatment, we have 1,460 samples. After outlier treatment, we have 1449 samples, which means 11 samples are lost due to outlier treatment

## 4.8.EDA 
It's now a good point to perform EDA to have a general visualization of the data.

### 4.8.1.Discrete/Categorical Variables v.s. Target

In [ ]:
# Function for visualizing discrete/categorical variables against SalePrice, boxplot
def box_plot2(x, y):
    ax = sns.boxplot(x=data[x], y=data[y], order=sorted_categories[x], palette = 'Set2', showmeans=True,
                     meanprops={"marker": "s", "markerfacecolor": "white", "markeredgecolor": "crimson"})
    ax.set_ylabel('')
    ax.set_title('{}'.format(x), fontsize=12, fontweight='bold', pad=5)
    ax.patch.set_edgecolor('black')
    ax.patch.set_linewidth(1.5)
    plt.title('{}'.format(x), fontsize=16)

# Assuming 'data' is your DataFrame containing the data
cat_cols = [cols for cols in data.columns if (cols != 'SalePrice') & (cols not in continous_features)]

# Calculate the mean of 'SalePrice' for each category in each categorical feature
medians = {}
for col in cat_cols:
    medians[col] = data.groupby(col)['SalePrice'].median()

# Sort the categorical features based on the mean of 'SalePrice'
sorted_categories = {col: sorted(data[col].unique(), key=lambda x: medians[col][x]) for col in cat_cols}

num_rows = (len(cat_cols) + 3) // 4
plt.figure(figsize=[20, num_rows * 6])
for i in range(len(cat_cols)):
    plt.subplot(num_rows, 4, i + 1)
    box_plot2(cat_cols[i], 'SalePrice')
plt.tight_layout()
plt.show()

### 4.8.2.Numerical feautures v.s. Target

In [ ]:
# Function for visualizing numerical variables against SalePrice, scatterplot
def scatter_plot(x):
    ax = sns.scatterplot(x=data[x], y=data['SalePrice'], alpha=0.35, linewidth=0)
    ax.set_title('{} vs SalePrice'.format(x), fontsize=12, pad=5)
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.patch.set_edgecolor('black')
    ax.patch.set_linewidth(1.5)

plt.figure(figsize=[18, 32])
for i in range(len(continous_features)):
    plt.subplot(13, 4, i + 1)
    scatter_plot(continous_features[i])
plt.tight_layout()
plt.show()

## 4.9.One-Hot Encoding
perform one-hot encoding for nominal features

In [ ]:
#find nominal features
nomin_features = data.select_dtypes(exclude = np.number).columns
nomin_features

In [ ]:
#perform one-hot encoding to nominal features
data_dummy = pd.get_dummies(data[nomin_features], drop_first = True)
data_dummy.head()

In [ ]:
#concat dummy features to data
data = pd.concat([data, data_dummy], axis = 1)

#drop nominal features
data = data.drop(nomin_features, axis = 1)

In [ ]:
data.shape

## 4.10.Dimensionality Reduction

### 4.10.1.Drop features with low variance
Since we now have up to 246 features after all previous operations, we may consider reduce dimensionality by drop features with rather small variance. 

In [ ]:
# Calculate variance of each feature
feature_var = data.var()

# Find features with variance close to 0
low_var_features = feature_var[feature_var < 0.001]

low_var_features

In [ ]:
low_var_features_list = low_var_features.index.tolist()
print(low_var_features_list)

In [ ]:
data = data.drop(low_var_features_list,axis=1)

### 4.10.2.Address Multicolinearity
Dimensionality reduction can also be achieved by picking among features with multicolinearity that has the highest correlation with target.

#### Analysis

In [ ]:
#Visualize Correlation 
Garage_SalePrice = ['GarageArea', 'GarageCars', 'GarageCond', 'GarageQual','GarageFinish_None', 'GarageType_None', 'GarageYrBlt_None', 'SalePrice']

plt.figure(figsize=(10,10))
g = sns.heatmap(data[Garage_SalePrice].corr(),annot=True,cmap="RdYlGn")

According to the correlation matrix, I want to develop an algorithm:
1. Compare between features with absolute correlation higher than 0.8.
In this case\
Group1: GarageArea & GarageCars\
Group2: GarageCond & GarageQual\
Group3: GarageFinish_None/GarageType_None/GarageYrBlt_None & GarageQual\
Group4: GarageFinish_None/GarageType_None/GarageYrBlt_None & GarageCond 
3. Pick features that has higher correlation with SalePrice:\
Group1: Garage_Cars\
Group2: GargeQual\
Group3: GarageQual\
Group4: GarageCond

Remove duplicates and repeat step1 & 2, features we need are: GarageCars, GarageQual

#### Develop Algorithm

In [ ]:
# Calculate the correlation matrix
corr_matrix = data.corr()

# Find features with correlation higher than 0.8
highly_correlated_features = set()

for i in range(len(corr_matrix.columns)):
    for j in range(i):
        if abs(corr_matrix.iloc[i, j]) > 0.8:
            colname_i = corr_matrix.columns[i]
            colname_j = corr_matrix.columns[j]
            highly_correlated_features.add((colname_i, colname_j))

highly_correlated_features

In [ ]:
# Calculate the correlation of each feature with the target
corr_with_target = data.corrwith(data['SalePrice'])

# Re-initialize the set of features to keep
keep_features = set()

for feature_pair in highly_correlated_features:
    feature1, feature2 = feature_pair
    
    # Compare the absolute correlation with target
    if abs(corr_with_target[feature1]) >= abs(corr_with_target[feature2]):
        keep_features.add(feature1)
    else:
        keep_features.add(feature2)

# Now we loop until no pair of kept features has a correlation above 0.8
while True:
    # Calculate the correlation matrix for the kept features
    corr_matrix_keep = data[keep_features].corr().abs()
    
    # Find pairs of features that have a correlation above 0.8
    corr_pairs = [(i, j) for i in range(len(corr_matrix_keep)) for j in range(i) if abs(corr_matrix_keep.iloc[i, j]) > 0.8]
    
    # If no such pairs are found, we can break the loop
    if not corr_pairs:
        break
    
    # Otherwise, we remove one feature from each pair
    for i, j in corr_pairs:
        feature1 = corr_matrix_keep.columns[i]
        feature2 = corr_matrix_keep.columns[j]
        
        # Compare the absolute correlation with target
        if abs(corr_with_target[feature1]) >= abs(corr_with_target[feature2]):
            # Check if the feature is still in the set before removing it
            if feature2 in keep_features:
                keep_features.remove(feature2)
        else:
            # Check if the feature is still in the set before removing it
            if feature1 in keep_features:
                keep_features.remove(feature1)

print('Features to keep are:')
print(keep_features)

I am happy to see my algorithm choose to keep 'GarageCars' & 'GarageQual' as it is supposed to. 

In [ ]:
# The features to drop are those that are in the original set of highly correlated features but not in the keep_features set
original_highly_correlated_features = set()
for pair in highly_correlated_features:
    original_highly_correlated_features.update(pair)

drop_highly_correlated_features = list(original_highly_correlated_features - keep_features)

print('Features to drop are:')
print(drop_highly_correlated_features)

In [ ]:
data = data.drop(drop_highly_correlated_features, axis =1)

## Export Cleaned Data

In [ ]:
# # export preprocessed data
# data.to_csv('preprocessed_data.csv', index=False)

# 5.Model Building

## 5.1.Feature Scalling

In [ ]:
# split into train and test
data_train, data_test = train_test_split(data, train_size=0.7, test_size = 0.3, random_state=42)

Perform feature scalling for numerical features except for ordinal_features_1, bin_features, and data_dummy.columns

In [ ]:
num_features = data.select_dtypes(include = np.number).columns

features_to_scal = [col for col in num_features if (col != 'SalePrice') & (col not in ordinal_features) 
                    & (col not in bin_features) & (col not in data_dummy.columns)]

print('Features_to_scal are:')
print(features_to_scal)

In [ ]:
#Scalling by standardization
scaler = StandardScaler()

data_train[features_to_scal] = scaler.fit_transform(data_train[features_to_scal])
data_test[features_to_scal] = scaler.transform(data_test[features_to_scal])

In [ ]:
# seperate features from target
X_train = data_train.drop('SalePrice', axis = 1)
y_train = data_train['SalePrice']

X_test = data_test.drop('SalePrice', axis = 1)
y_test = data_test['SalePrice']

## 5.2.Linear Regression

In [ ]:
#function for building linear regression model using statsmodel api
def build_model(cols):
    # Add a constant
    X_train_lm = sm.add_constant(X_train[cols], has_constant = 'add')
    # fitting model to data
    lr = sm.OLS(y_train, X_train_lm).fit()
    return lr

lr = build_model(X_train.columns)

X_train_lm = sm.add_constant(X_train, has_constant = 'add')
y_train_pred_lm = lr.predict(X_train_lm)
print('Train R2 score      : ', r2_score(y_train, y_train_pred_lm))
print('Train MAE           : ', round(mean_absolute_error(y_train, y_train_pred_lm),2))
print('Train RMSE          : ', round(mean_squared_error(y_train, y_train_pred_lm, squared = False),2), '\n')


X_test_lm = sm.add_constant(X_test, has_constant = 'add')
y_test_pred_lm = lr.predict(X_test_lm)
print('Test R2 score       : ', r2_score(y_test, y_test_pred_lm))
print('Test MAE           : ', round(mean_absolute_error(y_test, y_test_pred_lm),2))
print('Test RMSE          : ', round(mean_squared_error(y_test, y_test_pred_lm, squared = False),2), '\n')

At this point, the performance of linear regression is not ideal, so we need to perform further operations to improve the overall performance.

## 5.3.Logarithmic Transformation for Target
We have previously observed in 3.1.Target Distribution that our target is right-skewed. By logarithmic transformation, we make it more close to a normal distribution.

In [ ]:
y_train = np.log(y_train)
y_test = np.log(y_test)

## 5.4.Feature Selection using RFE
With our logarithmic transformed target, we use ‘Recursive Feature Elimination’ model to eliminate features with little contribution to model training. Meanwhile, n_features_to_select is set to be 100 but is flexible to change. 

In [ ]:
#importing utility
from sklearn.feature_selection import RFE

#Eliminating features using RFE
lm = LinearRegression()
selector = RFE(estimator = lm, n_features_to_select = 100)
selector.fit(X_train, y_train)

In [ ]:
#features selected bt rfe
cols = X_train.columns[selector.support_]
cols

In [ ]:
#updated training and test sets
X_train_new = X_train[cols]
X_test_new = X_test[cols]

## 5.5.Try with Different Linear Models

### 5.5.1.Lasso
Lasso uses L1 hyperparameter to penalize features with multicolinearity by assigning zero to predictive parameters. So, it helps us to further eliminate features with multicolinearity.

#### Parameter Optimization

In [ ]:
# list of alphas to tune

params = {'alpha': [0.00001, 0.0001, 0.001, 0.01, 0.05, 0.1, 
 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 2.0, 3.0, 
 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 20, 50, 100, 500, 1000 ]}


lasso = Lasso()

# cross validation
folds = 5
model_cv = GridSearchCV(estimator = lasso, 
                        param_grid = params, 
                        scoring= 'neg_mean_absolute_error', 
                        cv = folds, 
                        return_train_score=True,
                        verbose = 1)            

model_cv.fit(X_train_new, y_train) 

# Print best fit hyperparameter alpha
print(model_cv.best_params_)

#### Prediction

In [ ]:
# Lasso Model for best param
lasso = Lasso(alpha=0.001)
lasso.fit(X_train_new, y_train)

y_train_pred_lasso = lasso.predict(X_train_new)
print('Train R2 Score      : ', round(r2_score(y_train, y_train_pred_lasso),4))
print('Train MAE           : ', round(mean_absolute_error(y_train, y_train_pred_lasso),4))
print('Train RMSE          : ', round(mean_squared_error(y_train, y_train_pred_lasso, squared = False),4), '\n')

y_test_pred_lasso = lasso.predict(X_test_new)
print('Test R2 Score       : ', round(r2_score(y_test, y_test_pred_lasso),4))
print('Test MAE            : ', round(mean_absolute_error(y_test, y_test_pred_lasso),4))
print('Test RMSE           : ', round(mean_squared_error(y_test, y_test_pred_lasso, squared = False),4))

#### Predicted SalePrice

Since we performed logarithmatic transform to SalePrice, we will need to use use np.exp() to expand SalePrice to its original scale.

In [ ]:
y_test_pred_lasso

### 5.5.2.Rdige

#### Hyperparameter Optimization

In [ ]:
# list of alphas to tune
params = {'alpha': [0.0001, 0.001, 0.01, 0.05, 0.1, 
 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 2.0, 3.0, 
 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 20, 50, 100, 500, 1000 ]}

ridge = Ridge()

# cross validation
folds = 5
model_cv = GridSearchCV(estimator = ridge, 
                        param_grid = params, 
                        scoring= 'neg_mean_absolute_error',  
                        cv = folds, 
                        return_train_score=True,
                        verbose = 1)            
model_cv.fit(X_train_new, y_train) 

# Print best fit hyperparameter alpha
print(model_cv.best_params_)

#### Prediction

In [ ]:
# Ridge Model for best param
ridge = Ridge(alpha = 10.0)
ridge.fit(X_train_new, y_train)

y_train_pred_ridge = ridge.predict(X_train_new)
print('Train R2 Score      : ', round(r2_score(y_train, y_train_pred_ridge),4))
print('Train MAE           : ', round(mean_absolute_error(y_train, y_train_pred_ridge),4))
print('Train RMSE          : ', round(mean_squared_error(y_train, y_train_pred_ridge, squared = False),4), '\n')

y_test_pred_ridge = ridge.predict(X_test_new)
print('Test R2 Score       : ', round(r2_score(y_test, y_test_pred_ridge),4))
print('Test MAE            : ', round(mean_absolute_error(y_test, y_test_pred_ridge),4))
print('Test RMSE           : ', round(mean_squared_error(y_test, y_test_pred_ridge, squared = False),4))

### 5.5.3.Elastic Net

#### Hyperparameter Optimization

In [ ]:
# list of alphas to tune
params = {'alpha': [0.0001, 0.001, 0.01, 0.05, 0.1, 
 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 2.0, 3.0, 
 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 20, 50, 100, 500, 1000 ],
          'l1_ratio': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]}


en = ElasticNet()

# cross validation
folds = 5
model_cv = GridSearchCV(estimator = en, 
                        param_grid = params, 
                        scoring= 'neg_mean_absolute_error', 
                        cv = folds, 
                        return_train_score=True,
                        verbose = 1,
                        n_jobs=-1)            

model_cv.fit(X_train_new, y_train) 

# Print best fit hyperparameter alpha
print(model_cv.best_params_)

#### Prediction

In [ ]:
# ElasticNet Model for best param
en = ElasticNet(alpha=0.001, l1_ratio = 0.4)
en.fit(X_train_new, y_train)

y_train_pred_en = en.predict(X_train_new)
print('Train R2 Score      : ', round(r2_score(y_train, y_train_pred_en),4))
print('Train MAE           : ', round(mean_absolute_error(y_train, y_train_pred_en),4))
print('Train RMSE          : ', round(mean_squared_error(y_train, y_train_pred_en, squared = False),4), '\n')

y_test_pred_en = en.predict(X_test_new)
print('Test R2 Score       : ', round(r2_score(y_test, y_test_pred_en),4))
print('Test MAE            : ', round(mean_absolute_error(y_test, y_test_pred_en),4))
print('Test RMSE           : ', round(mean_squared_error(y_test, y_test_pred_en, squared = False),4))

### Model Evalution

In [ ]:
#Rdige
print('--Rdige--')
y_train_pred_ridge = ridge.predict(X_train_new)
print('Train R2 Score      : ', round(r2_score(y_train, y_train_pred_ridge),4))
print('Train MAE           : ', round(mean_absolute_error(y_train, y_train_pred_ridge),4))
print('Train RMSE          : ', round(mean_squared_error(y_train, y_train_pred_ridge, squared = False),4), '\n')

y_test_pred_ridge = ridge.predict(X_test_new)
print('Test R2 Score       : ', round(r2_score(y_test, y_test_pred_ridge),4))
print('Test MAE            : ', round(mean_absolute_error(y_test, y_test_pred_ridge),4))
print('Test RMSE           : ', round(mean_squared_error(y_test, y_test_pred_ridge, squared = False),4), '\n')

#Lasso
print('--Lasso--')
y_train_pred_lasso = lasso.predict(X_train_new)
print('Train R2 Score      : ', round(r2_score(y_train, y_train_pred_lasso),4))
print('Train MAE           : ', round(mean_absolute_error(y_train, y_train_pred_lasso),4))
print('Train RMSE          : ', round(mean_squared_error(y_train, y_train_pred_lasso, squared = False),4), '\n')

y_test_pred_lasso = lasso.predict(X_test_new)
print('Test R2 Score       : ', round(r2_score(y_test, y_test_pred_lasso),4))
print('Test MAE            : ', round(mean_absolute_error(y_test, y_test_pred_lasso),4))
print('Test RMSE           : ', round(mean_squared_error(y_test, y_test_pred_lasso, squared = False),4), '\n')

#Elastic Net
print('--Elastic Net--')
y_train_pred_en = en.predict(X_train_new)
print('Train R2 Score      : ', round(r2_score(y_train, y_train_pred_en),4))
print('Train MAE           : ', round(mean_absolute_error(y_train, y_train_pred_en),4))
print('Train RMSE          : ', round(mean_squared_error(y_train, y_train_pred_en, squared = False),4), '\n')

y_test_pred_en = en.predict(X_test_new)
print('Test R2 Score       : ', round(r2_score(y_test, y_test_pred_en),4))
print('Test MAE            : ', round(mean_absolute_error(y_test, y_test_pred_en),4))
print('Test RMSE           : ', round(mean_squared_error(y_test, y_test_pred_en, squared = False),4), '\n')

The 3 models have similar performance in terms of Test R2 Score(around 0.9), MAE (around 0.08), RMSE (around 0.11). Noticably, Ridge has the best performance -- it has the highest Test R2 score (0.9092), meaning that it has the best fitting effectiveness; it has the lowest MAE (0.0809), meaning that it generates the least errors on average; it also has the lowest RMSE (0.1148), meaning that it is least susceptible to noises and outliers. 

## 5.6.Important Features

### Coefficients of Ridge

In [ ]:
ridge_df = pd.DataFrame({'Features': ridge.feature_names_in_, 'Coefficients': ridge.coef_})
ridge_df = ridge_df.sort_values('Coefficients', ascending = False).reset_index().drop('index', axis = 1)
ridge_df

### Important Features

In [ ]:
important_df = ridge_df[abs(ridge_df.Coefficients) > 0.07].sort_values('Coefficients', ascending = False).reset_index().drop('index', axis = 1)
important_df

### Evaluate Multicolinearity by VIF

In [ ]:
#function to check for the VIF values of the feature variables. 
def get_vif(cols):
    # Create a dataframe that will contain the names of all the feature variables and their respective VIFs
    vif = pd.DataFrame()
    vif['Features'] = X_train[cols].columns
    vif['VIF'] = [variance_inflation_factor(X_train[cols].values, i) for i in range(X_train[cols].shape[1])]
    vif['VIF'] = round(vif['VIF'], 2)
    vif = vif.sort_values(by = "VIF", ascending = False)
    return vif

In [ ]:
get_vif(important_df['Features'])

All VIF values of important features are smaller than 5, which means multicolinearity is well addressed

### Visualization of factors with most significant impact on house price predication throughout the analysis

In [ ]:
#visualizing coefficients of most important features
important_df_sorted = important_df.sort_values(by='Coefficients')

plt.figure(figsize=[16, 8])
sns.barplot(x='Features', y='Coefficients', data = important_df_sorted, palette='RdYlGn')
plt.ylabel('Coefficient', fontsize=14)
plt.xlabel('')
plt.xticks(fontsize=12, rotation=90)
plt.show()

# Conclusion
I made an assumption that the missing value of 'Electrical' is a record mistake and impute it with mode. 

In our analysis, we used Ridge, Lasso, and ElasticNet as our predictive models. The 3 models have similar performance in terms of Test R2 Score(around 0.9), MAE (around 0.08), RMSE (around 0.11). Noticably, Ridge has the best performance -- it has the highest Test R2 score (0.9092), meaning that it has the best fitting effectiveness; it has the lowest MAE (0.0809), meaning that it generates the least errors on average; it also has the lowest RMSE (0.1148), meaning that it is least susceptible to noises and outliers. 

Model performance can be varied by adjusting following parameters:
1. Threshold values for removing outliers
2. Threshold value for removing low variance features(<0.001)
3. Threshold value for removing features with multicolinearity(>0.8)
4. Threshold value for RFE(n_features_to_select = 100)
5. Therehold value for hyperparameter optimization using GridSearchCV(folds = 5)

# 6.Prediction on Test

In [ ]:
# Load the data
test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')
df = test.copy()

## Preprocess Test Data

The process should be set in accordance to how we preprocess train.csv except for process that reduce samples, so we will not deal with outliers. Also, no need to drop any cols since it is taken care of by rfe and lasso. Therefore, the process we pick are: 4.2.Convert Data Type, 4.4.2.Treating Missing Values for Each Feature, 4.6.1.Ordinal Mapping, 4.6.2.Binary Mapping, and 4.9.One-Hot Encoding 

### Convert Data Type and Deal with Missing Values

In [ ]:
# 4.2.Convert Data Type
##############################################################################
# convert to object
df['MSSubClass'] = df['MSSubClass'].astype('object')

# 4.4.2.Treating Missing Values for Each Feature
############################################################################## 
#Impute with median
df['LotFrontage'].fillna(df['LotFrontage'].median(), inplace = True)

#impute with 'None'
df['Alley'].fillna('None', inplace = True)

# When missing values appears simultaneously, impute 'MasVnrType' with 'NA' and 'MasVnrArea' with 0.
df.loc[df['MasVnrType'].isnull() & df['MasVnrArea'].isnull(), ['MasVnrType','MasVnrArea']] = ['None',0.0]

# impute with 'None' for features not missing at random (TotalBsmtSF = 0)
cols = ['BsmtQual','BsmtCond','BsmtFinType1', 'BsmtExposure', 'BsmtFinType2']
df.loc[df['TotalBsmtSF'] == 0, cols] = df.loc[df['TotalBsmtSF'] == 0, cols].fillna('None')
#impute remaining missing values with mode
df['BsmtExposure'].fillna(df['BsmtExposure'].mode()[0], inplace = True)
df['BsmtFinType2'].fillna(df['BsmtFinType2'].mode()[0], inplace = True)

#Impute with mode
df['Electrical'].fillna(df['Electrical'].mode()[0], inplace = True)

#impute with 'None'
df['FireplaceQu'].fillna('None', inplace = True)

# impute with 'None' for features not missing at random (GarageArea = 0)
cols = ['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'GarageYrBlt']
df.loc[df['GarageArea'] == 0, cols] = df.loc[df['GarageArea'] == 0, cols].fillna('None')

#Feature Creation 
df['GarageYrBlt'] = df['GarageYrBlt'].apply(lambda x: 'None' if x == 'None' else ('New' if x > 1980 else 'Old'))

#impute with 'None'
df['PoolQC'].fillna('None', inplace = True)

#impute with 'None'
df['Fence'].fillna('None', inplace = True)

#impute with 'None'
df['MiscFeature'].fillna('None', inplace = True)

In [ ]:
# Check for missing values
total = df.isnull().sum().sort_values(ascending=False)
total = total[total > 0] 
percent = (df.isnull().sum() / df.shape[0] * 100).round(2).sort_values(ascending=False)
percent = percent[percent > 0]  

dtypes = df.dtypes[total.index]
result = pd.concat([total, percent, dtypes], axis=1, keys=['Total', 'Percent', 'Dtype'])
result

In [ ]:
# Getting list for remaining missing values
columns_with_missing_values = df.columns[df.isnull().any()].tolist()
print(columns_with_missing_values)

In [ ]:
# Find all continous columns
contin_columns = [cols for cols in columns_with_missing_values if cols in continous_features]
contin_columns

In [ ]:
# Fill continous features with median
df[contin_columns] = df[contin_columns].fillna(df[contin_columns].median())

In [ ]:
# Fill categorical features with mode
cate_columns = [cols for cols in columns_with_missing_values if cols not in contin_columns]
df[cate_columns] = df[cate_columns].fillna(df[cate_columns].mode().iloc[0])

In [ ]:
data.isnull().any().any()

### Feature Encoding

In [ ]:
# 4.6.1.Ordinal Mapping
############################################################################## 
ordinal_features_1 = ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'HeatingQC', 
                      'KitchenQual', 'FireplaceQu', 'GarageQual', 'GarageCond', 'PoolQC']

# conduct ordinal encoding for these features
for i in ordinal_features_1:
    df[i] = df[i].map({'Ex':9, 'Gd':7, 'TA':5, 'Fa':3, 'Po':2, 'None':0 })

# 4.6.2.Binary Mapping
############################################################################## 
# Applying the binary mapping to binary columns
df['Street'] = df['Street'].map({'Pave': 1, "Grvl": 0})
df['Utilities'] = df['Utilities'].map({'AllPub': 1, "NoSeWa": 0})
df['CentralAir'] = df['CentralAir'].map({'Y': 1, "N": 0})

# 4.9.One-Hot Encoding
############################################################################## 
#find nominal features
nomin_features = df.select_dtypes(exclude = np.number).columns

#perform one-hot encoding to nominal features
df_dummy = pd.get_dummies(df[nomin_features], drop_first = True)

#concat dummy features to data
df = pd.concat([df, df_dummy], axis = 1)

## Feature Scaling

In [ ]:
# Remembers that features_to_scal are features selected by RFE
df[features_to_scal] = scaler.transform(df[features_to_scal])

## Coefficients Extraction

In [ ]:
ridge_df = pd.DataFrame({'Features': ridge.feature_names_in_, 'Coefficients': ridge.coef_})
ridge_df = ridge_df.sort_values('Coefficients', ascending = False).reset_index().drop('index', axis = 1)
ridge_df

In [ ]:
# Extract features
features = ridge_df['Features']

# Convert 'features' into a set
features_set = set(features)

# Convert the feature names in 'df' into another set
df_features_set = set(df.columns)

# Find the features in 'features' that are not present in 'df'
features_not_in_df = list(features_set - df_features_set)

# Print the result
print("Features in 'features' but not in 'df':")
print(features_not_in_df)

In [ ]:
filtered_rows = ridge_df.loc[ridge_df['Features'].isin(features_not_in_df)]
filtered_rows

In [ ]:
ridge_df = ridge_df[~ridge_df['Features'].isin(features_not_in_df)]
ridge_df = ridge_df.reset_index().drop('index', axis = 1)
ridge_df

In [ ]:
# Step 1: Extracting feature names and corresponding linear regression coefficients from lasso_df
features = ridge_df['Features']
coefficients = ridge_df['Coefficients']
intercept = ridge.intercept_

# Step 2: Selecting corresponding features from df, note that index will be inherited to df_features
df_features = df[features]

# Step 3: Predicting 'SalePrice' using linear regression coefficients
# The prediction is the dot product of the coefficients and the selected features
predicted_SalePrice = df_features.dot(coefficients.values)+intercept

df_features.head()

In [ ]:
# Step 4: Adding the predicted 'SalePrice' to df
df['SalePrice'] = predicted_SalePrice

# Show the first few rows of the updated DataFrame
df.head()

## Final Prediction of SalePrice on Test.CSV

In [ ]:
#Reverse logarithmic transformation for target
df_result = df[['Id', 'SalePrice']]
df_result['SalePrice'] = np.exp(df_result['SalePrice'])
df_result.head()

## Export Results to CSV

In [ ]:
df_result.to_csv('yuanshan_submission.csv', index=False)

In [ ]:
df_result.shape